In [1]:
import numpy as np
import os

import torch
from torch.utils.data import DataLoader

from data_preparation import make_spec
from models.Inception import InceptionClassifier

In [2]:
train_dir = r"C:\Users\rafci\Desktop\tensorflow-speech-recognition-challenge\train"
test_dir = r"C:\Users\rafci\Desktop\tensorflow-speech-recognition-challenge\test"

if not os.path.exists(train_dir):
    train_dir = r"C:\Users\jakub\Desktop\PD\sem10\deep\tensorflow-speech-recognition-challenge\train"
    test_dir = r"C:\Users\jakub\Desktop\PD\sem10\deep\tensorflow-speech-recognition-challenge\test"

In [3]:


def get_test_list(test_dir):
    test_list  = []
     
    files = os.listdir(os.path.join(test_dir,'audio'))
    for _, f in enumerate(files):
        test_list.append(f)

    return test_list
def create_test_set(file_list, test_dir):
    X_array = np.zeros([len(file_list), 122, 85])
    file_names = []
    for ind, file in enumerate(file_list):
        X_array[ind] = make_spec(file,test_dir)

        file_names.append(file)
    return X_array, file_names

In [4]:
file_list = get_test_list(test_dir)
X_test, test_file_names=create_test_set(file_list,test_dir)
np.save(f"data/X_test.npy", np.expand_dims(X_test, -1)+1.3)
np.savetxt(f"data/test_file_names.txt", test_file_names, fmt='%s')

c:\Users\jakub\source\repos\transformers\data_preparation.py:46: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  D = librosa.amplitude_to_db(librosa.stft(sig[:FREQ], n_fft=512, hop_length=128, center=False),
c:\Users\jakub\source\repos\transformers\data_preparation.py:46: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  D = librosa.amplitude_to_db(librosa.stft(sig[:FREQ], n_fft=512, hop_length=128, center=False),


KeyboardInterrupt: 

In [ ]:
X_test = np.load("data/X_test.npy")
X_test = X_test.reshape((-1, X_test.shape[1], X_test.shape[2]))
print(X_test.shape)
test_file_names = np.loadtxt('data/test_file_names.npy', dtype=str)
print(test_file_names.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'data/X_test.npy'

In [ ]:
classes = os.listdir(train_dir+'/audio/')
NB_CLASSES = len(classes)
def convert_list_dict(lst):
    res_dct = {i: val for i, val in enumerate(lst)}
    return res_dct
         
classes_index = convert_list_dict(classes)
print(type(classes_index))
classes_index

<class 'dict'>


{0: 'bed',
 1: 'bird',
 2: 'cat',
 3: 'dog',
 4: 'down',
 5: 'eight',
 6: 'five',
 7: 'four',
 8: 'go',
 9: 'happy',
 10: 'house',
 11: 'left',
 12: 'marvin',
 13: 'nine',
 14: 'no',
 15: 'off',
 16: 'on',
 17: 'one',
 18: 'right',
 19: 'seven',
 20: 'sheila',
 21: 'silence',
 22: 'six',
 23: 'stop',
 24: 'three',
 25: 'tree',
 26: 'two',
 27: 'up',
 28: 'wow',
 29: 'yes',
 30: 'zero'}

In [ ]:
BATCH_SIZE = 64

X_test = torch.tensor(X_test)

from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, X, file_names):
        self.X = X.float().transpose(2, 1).to('cuda')
        self.file_names = file_names

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.file_names[idx]

test_dataset = CustomDataset(X_test, test_file_names)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
import csv
INPUT_SHAPE = (85, 122)

model = InceptionClassifier(INPUT_SHAPE, NB_CLASSES).to('cuda')
model.load_state_dict(torch.load('saved_models/inception_aug.pth'))
with open('outputs/output.csv', 'w', newline='') as csvfile:
    fieldnames = ['fname', 'label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    model.eval()
    with torch.no_grad():
        
        for inputs, file_names in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            for num,file_name in zip(predicted.cpu().numpy(),file_names):
                predicted_word = classes_index[num]
                
                writer.writerow({'fname': file_name, 'label': predicted_word})



In [ ]:
print(predicted)
words_list = [classes_index[num] for num in predicted.cpu().numpy()]
print(words_list)


tensor([14, 23, 13, 18, 21,  0, 14, 23, 21, 22], device='cuda:0')
['no', 'stop', 'nine', 'right', 'silence', 'bed', 'no', 'stop', 'silence', 'six']
